# Environment preparation - fetching dataset and library import

In [0]:
from google.colab import drive
import os
import sys
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from tqdm import tqdm
from collections import defaultdict
from scipy import misc
from sklearn.metrics import f1_score

In [0]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
! date && unzip -q /content/drive/'My Drive'/celeb_dataset.zip -d /content/  && date #unzip dataset into local machine

Tue Dec 18 18:37:12 UTC 2018
replace /content/celeb/Anno/list_landmarks_align_celeba.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
#create a directory for preprocessed TFRecord files
!ls /content/
!mkdir /content/preprocessed/
!ls /content/

celeb  drive  preprocessed  sample_data
mkdir: cannot create directory ‘/content/preprocessed/’: File exists
celeb  drive  preprocessed  sample_data


In [0]:
def peek_file(file_path, n_lines):
  """
  helper function to peek first n_lines of the file
  """
  try:
    n_lines = int(n_lines)
    assert(n_lines > 0)
  except ValueError:
    print("Number of lines argument must be integer-interpretable")
  except AssertionError:
    print("Number of lines must be positive")
    
  with open(file_path) as file:
    for line in range(n_lines):
      print(file.readline(), end="")
  return



# Settings

In [0]:
# path to downloaded dataset (unzipped)
DOWNLOADED_PATH = '/content/celeb' #@param {type:"string"}

# path to preprocessed dataset (tf records)
SAVED_DATASET = '/content/preprocessed/' #@param {type:"string"}

TRAIN_FILENAME = 'train.tfrecords' #@param {type:"string"}
VAL_FILENAME = 'val.tfrecords' #@param {type:"string"}
TEST_FILENAME = 'test.tfrecords' #@param {type:"string"}

ANNO_PATH = os.path.join(DOWNLOADED_PATH, 'Anno',  'list_attr_celeba.txt') #file that indicates to which classes an image belongs to 
IMG_PATH = os.path.join(DOWNLOADED_PATH, 'Img') #path to images directory
EVAL_PATH = os.path.join(DOWNLOADED_PATH, 'Eval', 'list_eval_partition.txt') #file that indicates how to split the dataset into training, validation and testing sets

# tensorflow tf records dataset settings
DECODE_PARALLEL_CALLS = 260 #@param {type:"slider", min:20, max:1000, step:1}
SHUFFLE_BUFFER_SIZE = 100 #@param {type:"slider", min:20, max:1000, step:1}
PREFETCH_BUFFER_SIZE = 100 #@param {type:"slider", min:20, max:1000, step:1}

BATCH_SIZE = 32 #@param {type:"slider", min:0, max:1000, step:1}

EPOCHS = 3 #@param {type:"slider", min:0, max:1000, step:1}


COUNT_LIMIT = None #@param {type:"raw"}

LEARNING_RATE =  0.00001 #@param {type:"slider", min: 0.00001, max:1, step: 0.00001}





# Decoding TFRecords

In [0]:
def dataset_wrapper(session, dataset):
    """
    Wraps TensorFlow dataset into generator.
    :param session:
    :param dataset: tf.data.Dataset object
    """
    iterator = dataset.make_one_shot_iterator()
    next_element = iterator.get_next()
    while True:
        try:
            data = session.run(next_element)
            yield data
        except tf.errors.OutOfRangeError:
            break
            
def _decode_record(record):
    """
    Decodes record from bytes to arrays.
    """
    features_dict = {
        'image': tf.FixedLenFeature((), tf.string),
        'tags': tf.FixedLenFeature((), tf.string),
    }

    features = tf.parse_single_example(record, features=features_dict)
    #data had been written as uint8 to save up space...
    image = tf.decode_raw(features['image'], tf.uint8)
    image = tf.reshape(image, shape=(218, 178, 3))
    #.. but we need more precison on calculation
    image = tf.cast(image, tf.float32)
    image = image / 255.0 #normalize
    tags = tf.decode_raw(features['tags'], tf.uint8)

    return {
        'image': image,
        'tags': tags
    }

In [0]:
def make_dataset(dataset_path, batch_size=128, shuffle=False, epochs=1, count_limit=None):
    """
    Creates dataset pipeline.
    :param count_limit:
    :return: tf.data.TFRecordDataset object
    """
    # Read a tf record file. This makes a dataset of raw TFRecords
    dataset = tf.data.TFRecordDataset([dataset_path])

    if count_limit is not None:
        dataset = dataset.take(count_limit)

    # Can be parallelized!
    dataset = dataset.map(_decode_record, num_parallel_calls=DECODE_PARALLEL_CALLS)

    dataset = dataset.repeat(epochs)

    if shuffle:
        # Shuffle the dataset
        dataset = dataset.shuffle(buffer_size=SHUFFLE_BUFFER_SIZE)

    # Batch the dataset so that we get batch_size examples in each batch.
    # Remember each item in the dataset is a dict of tensors,
    # we need to specify padding for each tensor seperatly
    dataset = dataset.batch(batch_size)

    # Async loading, must-have!
    dataset = dataset.prefetch(buffer_size=PREFETCH_BUFFER_SIZE)

    return dataset

# Writting to TFRecords

In [0]:
peek_file(EVAL_PATH, 20)

000001.jpg 0
000002.jpg 0
000003.jpg 0
000004.jpg 0
000005.jpg 0
000006.jpg 0
000007.jpg 0
000008.jpg 0
000009.jpg 0
000010.jpg 0
000011.jpg 0
000012.jpg 0
000013.jpg 0
000014.jpg 0
000015.jpg 0
000016.jpg 0
000017.jpg 0
000018.jpg 0
000019.jpg 0
000020.jpg 0


In [0]:
def _load_splits():
    """
    Load split for train/val/test dataset.
    To be implemented by students.
    :return:
    """
    splits = defaultdict(lambda: list())

    with open(EVAL_PATH, mode='rt', encoding='utf-8') as file_:
        for line in file_:
            file_name, class_value = line.strip().split(' ')
            assert(class_value in ['0', '1', '2'])
            splits[class_value].append(file_name)

    return dict(splits)

In [0]:
peek_file(ANNO_PATH, 20)

202599
5_o_Clock_Shadow Arched_Eyebrows Attractive Bags_Under_Eyes Bald Bangs Big_Lips Big_Nose Black_Hair Blond_Hair Blurry Brown_Hair Bushy_Eyebrows Chubby Double_Chin Eyeglasses Goatee Gray_Hair Heavy_Makeup High_Cheekbones Male Mouth_Slightly_Open Mustache Narrow_Eyes No_Beard Oval_Face Pale_Skin Pointy_Nose Receding_Hairline Rosy_Cheeks Sideburns Smiling Straight_Hair Wavy_Hair Wearing_Earrings Wearing_Hat Wearing_Lipstick Wearing_Necklace Wearing_Necktie Young 
000001.jpg -1  1  1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1  1  1 -1  1 -1 -1  1 -1 -1  1 -1 -1 -1  1  1 -1  1 -1  1 -1 -1  1
000002.jpg -1 -1 -1  1 -1 -1 -1  1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1  1 -1  1 -1 -1  1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1  1
000003.jpg -1 -1 -1 -1 -1 -1  1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1  1  1 -1 -1  1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1  1
000004.jpg -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1  1 -1 -1 -1 -1  1 -1  1 -1  1  1 -1  1
0000

In [0]:
def _load_tags():
    """
    Load tags (classes) for images.
    To be implemented by students.
    :return:
    """
    tags_result = dict()
    with open(ANNO_PATH, mode='rt', encoding='utf-8') as file_:
        next(file_)
        tag_names = [tag.lower() for tag in next(file_).strip().split(' ')]
        for line in file_:
            parsed_line = line.strip().replace('  ', ' ').split(' ')
            file_name = parsed_line[0]
            tags = np.array([int(tag) for tag in parsed_line[1:]])
            tags = tags > 0.5
            tags = tags.astype(np.uint8)
            tags_result[file_name] = tags

    return tags_result, tag_names

In [0]:
def _bytes_feature(value):
    """
    Helper function for feature encoding.
    :param value:
    :return:
    """
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))

def _write_records(dataset_filename, image_filename_iterator, tags):
    """
    Writes images and tags to file.
    :param dataset_filename:
    :param image_filename_iterator: iterable containing filenames
    :param tags: dictionary containing pairs filename: tags
    :return:
    """
    with tf.python_io.TFRecordWriter(dataset_filename) as writer:
        for image_filename in tqdm(iterable=image_filename_iterator,
                                   desc='writting {} dataset'.format(dataset_filename)):
            tag = tags[image_filename]
            image = misc.imread(os.path.join(IMG_PATH, image_filename.replace('.jpg', '.png')))

            # store as uint8
            image = image.astype(np.uint8)
            feature = {
                'image': _bytes_feature([image.tostring()]),
                'tags': _bytes_feature([tag.tostring()]),
            }
            example = tf.train.Example(features=tf.train.Features(feature=feature))
            writer.write(example.SerializeToString())
    sys.stdout.flush()

In [0]:
def _create_dataset(train_filename, val_filename, test_filename):
    """
    Creates dataset in target location.
    To be implemented by students
    :param train_filename:
    :param val_filename:
    :param test_filename:
    """
    splits = _load_splits()
    tags, _ = _load_tags()

    _write_records(train_filename, splits['0'], tags)
    _write_records(val_filename, splits['1'], tags)
    _write_records(test_filename, splits['2'], tags)

In [0]:
def maybe_create_dataset():
    """
    Function creates TensorFlow dataset if it does not exist.
    :return: (train_filename, val_filename, test_filename)
    """
    train_filename = os.path.join(SAVED_DATASET, TRAIN_FILENAME)
    val_filename = os.path.join(SAVED_DATASET, VAL_FILENAME)
    test_filename = os.path.join(SAVED_DATASET, TEST_FILENAME)

    if not all([os.path.exists(filename)
                for filename
                in [train_filename, val_filename, test_filename]]):
        _create_dataset(train_filename, val_filename, test_filename)
    return train_filename, val_filename, test_filename

#Model

In [0]:
    def report_parameters(scope=None):
        """
        Reports names, shape and number of parameters.
        :param scope: string, scope to report, defaults to global values
        :return:
        """
        denses = 0
        other = 0
        print('Summary')
        for parameter_tensor in tf.trainable_variables(scope=scope):
            total = np.prod(parameter_tensor.shape).value
            print('Name: {}, Shape: {}, Total: {}'.format(
                parameter_tensor.name, parameter_tensor.shape, total))

            if 'dense' in parameter_tensor.name:
                denses += total
            else:
                other += total

        print('')
        print('Feature Extractor: {} -> {:.2f}%'.format(other, 100*other/(other+denses)))
        print('Predictor: {} -> {:.2f}%'.format(denses, 100*denses/(other+denses)))
        print('')

In [0]:
def get_class_weights():
  
  
    """
    Creates weights for classes based on training set.
    :return: numpy array [num_classes,]
    """
    tags, _ = _load_tags()
    splits = _load_splits()

    weights = np.array([tags[name] for name in splits['0']])
    weights = weights.mean(axis=0)
    weights = 1.0 / weights

    return weights


In [0]:
class AlexNet:
    def __init__(self, input_shape, num_outputs, class_weights=None, create_loss=True, create_metrics=True):
        """
        Constructor for AlexNet, creates model, loss and metrics based on input and output shapes.
        :param input_shape: tuple, (width, height, channels)
        :param num_outputs: integer, number of classes in prediction
        :param class_weights: array, containing weights for positive classes, length equals to num_outputs
        :param create_loss: bool, indicates if loss should be created
        :param create_metrics: bool, idicates if tensorboard metrics should be created
        """
        self._input_shape = list(input_shape)
        self._num_outputs = num_outputs
        self._class_weights = class_weights

        with tf.variable_scope('alex_net'):
            self.p_image, self.p_tags, self.p_training = self._create_placeholders()
            self.o_tags = self._create_model()
            if create_loss:
                self.o_loss = self._create_loss()
            else:
                self.o_loss = tf.no_op()

            if create_metrics:
                self._create_metrics()



    def _create_placeholders(self):
        """
        Function creates placeholders for data. After running, 3 placeholders should be created:
        p_image for images, p_tags for expected outputs, p_training to indicate training phase.
        :return: (p_image, p_tags, p_training)
        """
        with tf.variable_scope('placeholders'):
            p_image = tf.placeholder(
                dtype=tf.float32,
                shape=[None] + self._input_shape,
                name='p_image'
            )

            p_tags = tf.placeholder(
                dtype=tf.float32,
                shape=[None, self._num_outputs],
                name='p_tags'
            )

            p_training = tf.placeholder(
                dtype=tf.bool,
                shape=(),
                name='p_training',
            )

        return p_image, p_tags, p_training


    def _augment(self, images):
        """
        Function randomly augments images during training.
        :param images: tensor (batch, width, height, channels), contains images in current batch
        :return: augmented images with the same shape as input
        """
        images = tf.image.random_flip_left_right(images)
        images = tf.image.random_hue(images, 0.01)
        images = tf.image.random_saturation(images, 0.2, 1.8)
        images = tf.image.random_brightness(images, 0.2)
        images = tf.minimum(images, 1.0)
        images = tf.maximum(images, 0.0)
        return images

    def _create_model(self):
       
      
      
        """
        Function that creates model based on placeholders in self.
        :return: tensor, logits for outputs (batch, num_outputs)
        """
        with tf.variable_scope('augmentation'):
            layer_input = tf.cond(
                self.p_training,
                true_fn=lambda: self._augment(self.p_image),
                false_fn=lambda: tf.identity(self.p_image),
            )

        with tf.variable_scope('conv1'):
            layer_input = tf.layers.conv2d(
                inputs=layer_input,
                filters=96,
                kernel_size=(11, 11),
                strides=(4, 4),
                padding='VALID',
                use_bias=True,
            )
            layer_input = tf.nn.relu(layer_input)

        with tf.variable_scope('norm1'):
            layer_input = tf.nn.local_response_normalization(
                input=layer_input,
                depth_radius=2,
                alpha=1e-05,
                beta=0.75,
                bias=1.0,
            )

        with tf.variable_scope('pool1'):
            layer_input = tf.layers.max_pooling2d(
                inputs=layer_input,
                pool_size=(3, 3),
                strides=(2, 2),
                padding='VALID',
            )

        with tf.variable_scope('conv2'):
            layer_input = tf.layers.conv2d(
                inputs=layer_input,
                filters=256,
                kernel_size=(5, 5),
                strides=(1, 1),
                padding='SAME',
                use_bias=True,
            )
            layer_input = tf.nn.relu(layer_input)

        with tf.variable_scope('norm2'):
            layer_input = tf.nn.local_response_normalization(
                input=layer_input,
                depth_radius=2,
                alpha=1e-05,
                beta=0.75,
                bias=1.0,
            )

        with tf.variable_scope('pool2'):
            layer_input = tf.layers.max_pooling2d(
                inputs=layer_input,
                pool_size=(3, 3),
                strides=(2, 2),
                padding='VALID',
            )

        with tf.variable_scope('conv3'):
            layer_input = tf.layers.conv2d(
                inputs=layer_input,
                filters=384,
                kernel_size=(3, 3),
                strides=(1, 1),
                padding='SAME',
                use_bias=True,
            )
            layer_input = tf.nn.relu(layer_input)

        with tf.variable_scope('conv4'):
            layer_input = tf.layers.conv2d(
                inputs=layer_input,
                filters=384,
                kernel_size=(3, 3),
                strides=(1, 1),
                padding='SAME',
                use_bias=True,
            )
            layer_input = tf.nn.relu(layer_input)

        with tf.variable_scope('conv5'):
            layer_input = tf.layers.conv2d(
                inputs=layer_input,
                filters=256,
                kernel_size=(3, 3),
                strides=(1, 1),
                padding='SAME',
                use_bias=True,
            )
            layer_input = tf.nn.relu(layer_input)

        with tf.variable_scope('pool5'):
            layer_input = tf.layers.max_pooling2d(
                inputs=layer_input,
                pool_size=(3, 3),
                strides=(2, 2),
                padding='VALID',
            )

        with tf.variable_scope('flatten'):
            layer_input = tf.reshape(
                tensor=layer_input,
                shape=[-1, np.prod(layer_input.shape[1:])],
            )

        with tf.variable_scope('fc6'):
            layer_input = tf.layers.dense(
                inputs=layer_input,
                units=1*1024,
                use_bias=True,
            )
            layer_input = tf.nn.relu(layer_input)

        with tf.variable_scope('dropout6'):
            layer_input = tf.layers.dropout(
                inputs=layer_input,
                rate=0.5,
                training=self.p_training,
            )

        with tf.variable_scope('fc7'):
            layer_input = tf.layers.dense(
                inputs=layer_input,
                units=1*1024,
                use_bias=True,
            )
            layer_input = tf.nn.relu(layer_input)

        with tf.variable_scope('dropout7'):
            layer_input = tf.layers.dropout(
                inputs=layer_input,
                rate=0.5,
                training=self.p_training,
            )

        with tf.variable_scope('fc8'):
            layer_input = tf.layers.dense(
                inputs=layer_input,
                units=self._num_outputs,
                use_bias=True,
            )

        return layer_input

    def _create_loss(self):
        
      
        """
        Adds cross-entropy loss function to the graph. If self.class_weights 
        is not None, cross-entropy is wieghted by self.class_weights
        :return: reference to loss function operation
        """
        
        with tf.variable_scope('loss'):
            if self._class_weights is not None:
                o_loss = tf.nn.weighted_cross_entropy_with_logits(
                    targets=self.p_tags,
                    logits=self.o_tags,
                    pos_weight=self._class_weights,
                )
            else:
                o_loss = tf.nn.sigmoid_cross_entropy_with_logits(
                    labels=self.p_tags,
                    logits=self.o_tags,
                )
                
        return tf.reduce_mean(o_loss)

    def _create_metrics(self):
        pass

# Metrics

In [0]:
def calculate_per_class_fscore(y_true, y_pred):
  
    """
    Calculates per class fscore. Applied every epoch.
    :param y_true: np.array of ground truth (binary). Shape: (examples in dataset, labels)
    :param y_pred: np.array of prediction per class (binary). Shape: (examples in dataset, labels)
    :return: np.array of f1 score per class. Shape: (labels, ) 
    """
    return [f1_score(y_true[:, num_class], y_pred[:, num_class])
            for num_class in range(y_true.shape[1])]

#Main pipeline

In [0]:
train_filename, val_filename, test_filename = maybe_create_dataset()

writting /content/preprocessed/train.tfrecords dataset:   0%|          | 0/162770 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
writting /content/preprocessed/train.tfrecords dataset: 100%|██████████| 162770/162770 [13:37<00:00, 199.10it/s]
writting /content/preprocessed/val.tfrecords dataset: 100%|██████████| 19867/19867 [01:41<00:00, 195.24it/s]
writting /content/preprocessed/test.tfrecords dataset: 100%|██████████| 19962/19962 [01:41<00:00, 196.79it/s]


In [0]:
tf.reset_default_graph()
with tf.variable_scope('faces'):
    
    faces_net = AlexNet((218, 178, 3), 40, create_loss=True, create_metrics=False)

In [0]:

def epoch_loop(session, model, dataset, training_op=None):
    """
    Basic train/test loop. Runs model on given dataset.
    :param session: session to the graph 
    :param model: AlexNet object
    :param dataset: dataset to train/validate on
    :param training_op: minimize operation if training, None if validating/testing
    :return: (total_loss, per_class_fscores)
    """
    do_train = training_op is not None # pass this to a placeholder
    training_op = tf.no_op() if training_op is None else training_op
    
    #TASK 3

    total_loss = list() # list of loss function after each batch
    tags_pred = list() #list of predicted tags
    tags_true = list() # list of ground truth
    per_class_fscores = list()
    
    iterator = dataset.make_initializable_iterator()
    session.run(iterator.initializer)

    next_batch = iterator.get_next()

    session.run(tf.global_variables_initializer())

    
    for data in tqdm(dataset_wrapper(session, dataset)):
      result = session.run(model.o_tags, feed_dict={model.p_image: data['image'],
                  model.p_tags: data['tags'],
                  model.p_training: True,})
      print(tf.math.sigmoid(result).eval())
      break;

    return total_loss, per_class_fscores

In [0]:
def run_training(model, train_filename, val_filename, learning_rate):
    """
    Performs complete training.
    :param model:properly build and adapted AlexNet for CelebFaces dataset
    :param train_filename: path to train TFRecords file
    :param val_filename: path to validation TFRecord file
    :param test_filename: path to test TFRecord file
    :param run:
    :return:
    """
#     with tf.variable_scope('optimization'):

        # TASK 1 - Add optimizer on loss function
        # DONE 18.12.2018 20:04
#         training_op = tf.train.AdamOptimizer(learning_rate).minimize(model._create_loss())
        #init_op = tf.initialize_all_variables()
        
        
    with tf.Session() as session:
      
        training_op = tf.train.AdamOptimizer(learning_rate).minimize(model._create_loss())
        # TASK 2 - initialize variables of the model; for every epoch, create 2 datasets: train and validation
        # using make_dataset function
        
        #epoch_loop(session, model, dataset, training_op=None)
        #make_dataset(dataset_path, batch_size=128, shuffle=False, epochs=1, count_limit=None)
        for epoch in range(EPOCHS):
       
            train_dataset = make_dataset(train_filename, batch_size=128, shuffle=True, epochs=1, count_limit=10)
            validation_dataset = make_dataset(val_filename, batch_size=128, shuffle=False, epochs=1, count_limit=10)
            
            train_loss, train_fscores = epoch_loop(session, model, train_dataset, training_op) # feed it with proper arguments
            val_loss, val_fscores = epoch_loop(session, model, validation_dataset) # feed it with proper arguments


            print('Train: Loss: {} FScore: {}'.format(train_loss, train_fscores.mean()))
            print('Val: Loss: {} FScore: {}'.format(val_loss, val_fscores.mean()))
#             print('Test: Loss: {} FScore: {}'.format(test_loss, test_fscores.mean()))



In [0]:
run_training(faces_net, train_filename, val_filename, learning_rate = 0.1)

0it [00:00, ?it/s]

[[0.50808156 0.5026448  0.49377736 0.4971764  0.4770981  0.47324893
  0.5073217  0.5029614  0.49981833 0.48253897 0.49307847 0.4988124
  0.4934033  0.5069897  0.51285833 0.48443902 0.5025996  0.47642162
  0.4831494  0.517575   0.46190384 0.48143837 0.49820426 0.4990758
  0.4989071  0.49988332 0.5102258  0.49327007 0.51089865 0.50244665
  0.48973373 0.5051826  0.5143566  0.491346   0.49245965 0.5080527
  0.49675146 0.4985261  0.4702353  0.50140005]
 [0.4887118  0.5212405  0.47877914 0.49065885 0.476075   0.5046369
  0.5063266  0.48717445 0.49022993 0.4919198  0.50566864 0.5041952
  0.50187135 0.51737046 0.4829657  0.49669722 0.50947315 0.4929175
  0.49351773 0.50930834 0.48927945 0.48879638 0.48862827 0.49937707
  0.50314057 0.51699954 0.5081155  0.4958262  0.5026361  0.47595522
  0.49982482 0.5036967  0.50963134 0.50955945 0.4807838  0.5066466
  0.49774298 0.4923181  0.49494675 0.50025713]
 [0.5070104  0.5172233  0.501683   0.50183916 0.48794943 0.5083543
  0.5058552  0.5041571  0.5186


0it [00:00, ?it/s]

[[0.48041797 0.49777666 0.50768864 0.51756203 0.5014831  0.49969488
  0.49314305 0.4864622  0.4926691  0.5147382  0.50645053 0.488077
  0.50730723 0.50333077 0.50134546 0.5014474  0.4679657  0.48614928
  0.4951597  0.50398046 0.5036476  0.4733189  0.5027347  0.520713
  0.496088   0.51536113 0.49741197 0.49893773 0.50636894 0.5016812
  0.49556258 0.50043106 0.4876575  0.49063542 0.48522952 0.4982153
  0.48631123 0.4978211  0.5073012  0.49145856]
 [0.5048896  0.5129661  0.50020593 0.5043577  0.49582517 0.50857913
  0.49413002 0.463412   0.49381375 0.48630852 0.5073737  0.5142969
  0.50656766 0.5113311  0.520004   0.48548192 0.50483197 0.4834439
  0.49830425 0.5044077  0.5075693  0.48100036 0.4941058  0.51333535
  0.503854   0.5031838  0.49679363 0.5069006  0.50782716 0.49504894
  0.50361    0.5089896  0.5091132  0.48508996 0.5003667  0.4831671
  0.47158778 0.4671948  0.5052024  0.5020227 ]
 [0.5021665  0.5097747  0.49573362 0.5010353  0.49840283 0.48977515
  0.49029383 0.4883808  0.50913

AttributeError: ignored